### 自定义广告项目IDF 语料库

由于广告行业的特殊性，不适合使用通用的IDF语料库，所以用自定义一个语料库逆文档。

用了2019年至今，国内8000多个广告项目的文本资料来构建自定义的IDF文本语料库

In [1]:
import pandas as pd
import numpy as np
import requests
from pandas import DataFrame

In [2]:
df1= pd.read_csv('ads_content_7224.csv')
df2=pd.read_csv('socialbeta_all.csv')

In [3]:
content_ls=df1['ad_content'].to_list()
_=df2['content'].to_list()
content_ls +=_
df_content=pd.DataFrame(content_ls,columns=['content'])
df_content['content'].astype('str')
df_content.dropna(axis=0, how='any', inplace=True)

In [4]:
df_content.shape

(8387, 1)

In [5]:
import re
import math
import jieba
import os

In [6]:
# 对文章进行分词
#载入自定义的词典
jieba.load_userdict('/Users/julia/learndata/dic_jieba/jiebaDict.txt')

df_content['content'] = df_content['content'].apply(jieba.lcut)
#过滤停用词\长度小于1的词\非中文词
pattern = re.compile(u'[\u4e00-\u9fa5]+')
stopwords = [line.strip().encode('utf-8').decode('utf-8') for line in open('/Users/julia/learndata/dic_jieba/cn_stopwords.txt').readlines()]
# 判断字符串是不是纯中文
df_content['content']= df_content['content'].apply(
    lambda cut_words: [word for word in cut_words if word not in stopwords and len(word) > 1 and pattern.search(word)])

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/fy/71rk0bjs7rv6r71b4g21lqcc0000gn/T/jieba.cache
Loading model cost 0.741 seconds.
Prefix dict has been built successfully.


In [7]:
# 对每篇文章结果进行去重,并获取所有的分词结果
content_cut = df_content['content'].apply(lambda s: list(set(s)))
content_word=[]
for i in content_cut.index:
    content_word += content_cut[i]
content_word = set(content_word)


#构造字典，用于存放包含某个词汇文档的数量
path_num = os.path.join(r"/Users/julia/Desktop/", 'num_dict.txt')

try:
    # 先读取上次结果， 若不存在则重新构造
    dict_df = pd.read_csv(path_num, encoding='gbk', sep=' ', header=None, names=['word', 'idf'])
    dic = dict(zip(dict_df['word'], dict_df['idf']))
except Exception as e:
	# 新构造的字典初始值全为0
    dic = dict(zip(content_word, [0] * len(content_word)))

# 计算IDF值
# 1.更新出现每个词文章的数目
for i in content_cut.index:
    for word in content_cut[i]:
        try:
            dic[word] += 1
        except:
            dic[word] = 1
# 2.保存次数，用于下一次的增量更新
file = open(path_num, 'w', encoding='utf-8')
for key, value in dic.items():
    try:
        text = key + ' ' + str(value) + '\n'
        file.write(text)
    except Exception as e:
        print(e)
file.close()

# 3.计算IDF值
path_idf = os.path.join(r"/Users/julia/Desktop/", "idf_dict.txt")
n = df_content.shape[0]  # 文章总数
idf_dic = {key: math.log(n / value, 10) for key, value in dic.items()}
 # 保存，添加到结巴的提取关键词词库
file = open(path_idf, 'w', encoding='utf-8')
for key, value in idf_dic.items():
    try:
        text = key + ' ' + str(value) + '\n'
        file.write(text)
    except Exception as e:
        print(e)
file.close()

### 关键词提取效果测试

从测试结果看，还是很明显能看出不同广告项目间的差别，比如：

- 有的项目关键词体现创意，大多是名/动词，构建意象，言之有物

- 有的项目关键词堆砌营销术语，云山雾罩，相对比较空洞乏味

- 有的项目关键词反复强调项目广告主或项目代理商，展示强大的背景

In [3]:
import jieba.analyse

In [4]:
df= pd.read_csv('ads_content_7224.csv')
test1= df.iloc[180,0]
test2= df.iloc[235,0]
test3= df.iloc[21,0]

In [6]:
content = test1

jieba.analyse.set_stop_words('/Users/julia/learndata/dic_jieba/cn_stopwords.txt') 

jieba.analyse.set_idf_path('/Users/julia/learndata/dic_jieba/idf_dict.txt')

res = jieba.analyse.extract_tags(content, topK=20, withWeight=True)

for word,weight in res:
    print('%s %.6f'%(word, weight))

长白山 0.194842
新风 0.175790
空调 0.082236
空气 0.068849
呼吸 0.062611
增氧 0.039733
换气 0.034901
新鲜空气 0.031853
风景区 0.026176
长白山天池 0.018342
广州 0.016565
自然 0.015927
休息室 0.015926
家电 0.015146
带给 0.014737
城市 0.014237
跨界 0.012472
用户 0.011472
压抑 0.010976
组长 0.010863


In [7]:
content = test2

jieba.analyse.set_stop_words('/Users/julia/learndata/dic_jieba/cn_stopwords.txt') 

jieba.analyse.set_idf_path('/Users/julia/learndata/dic_jieba/idf_dict.txt')

res = jieba.analyse.extract_tags(content, topK=20, withWeight=True)

for word,weight in res:
    print('%s %.6f'%(word, weight))

康养 0.090259
平安 0.038285
高客 0.015928
商业 0.015130
养老 0.015034
至善 0.014706
价值体系 0.014706
品牌 0.014198
战略 0.013852
高端 0.013468
价值 0.013203
真正 0.011782
净值 0.011700
复杂 0.010779
构建 0.010645
量变 0.010619
New 0.010619
虚怀若谷 0.009804
竞争 0.009672
之道 0.009178


In [8]:
content = test3

jieba.suggest_freq('华Sir',tune=True)

jieba.analyse.set_stop_words('/Users/julia/learndata/dic_jieba/cn_stopwords.txt') 

jieba.analyse.set_idf_path('/Users/julia/learndata/dic_jieba/idf_dict.txt')

res = jieba.analyse.extract_tags(content, topK=20, withWeight=True)

for word,weight in res:
    print('%s %.6f'%(word, weight))

买药 0.080098
美团 0.049181
24 0.035481
胜加 0.024679
真诚 0.018498
大众 0.012539
华Sir 0.011827
小时 0.010003
服务 0.009808
夜间 0.008415
用药 0.007509
品牌 0.007413
用到 0.007211
值得 0.007096
真的 0.006838
知道 0.006636
药店 0.006610
情感 0.006607
价值 0.006536
短片 0.006405


### 比较文本余弦相似度

广告是瞄准人性的生意，方向一致，就算不同的项目也经常有类似之处，那到底差别有多大呢。

同一类型的广告主（如电商），同一营销节点（如春节），不同代理公司出品的项目在多大程度上相似。

同一代理公司，在面对不同的广告主时，给出的项目究竟是量身定制还是一招鲜吃遍天呢？

尝试了三种方法，利用余弦相似度，比较不同广告项目文本的相似程度。

In [9]:
#基于TF-IDF计算两个文本的余弦相似度

#分别取关键词并生成向量
def get_cos_similarity_tfidf(s1,s2):
    
    jieba.analyse.set_stop_words('/Users/julia/learndata/dic_jieba/cn_stopwords.txt') 
    jieba.analyse.set_idf_path('/Users/julia/learndata/dic_jieba/idf_dict.txt')
    
    res1 = jieba.analyse.extract_tags(s1, topK=20, withWeight=False)
    res2 = jieba.analyse.extract_tags(s2, topK=20, withWeight=False)
    key_word1=[]
    key_word2=[]
    
    for word in res1:
        key_word1.append(word)
    for word in res2:
        key_word2.append(word)
        
#     print(key_word1)
#     print(key_word2)
    
    #取关键词的并集
    key_word= list(set(key_word1+key_word2))
    
    #两个储存用的空向量
    key_word_vector1= np.zeros(len(key_word))
    key_word_vector2= np.zeros(len(key_word))
    
    #计算词频
    for i in range(len(key_word)):
        
        for j in range(len(key_word1)):
            if key_word[i]==key_word1[j]:
                key_word_vector1[i]+=1
                       
        for k in range(len(key_word2)):
            if key_word[i]==key_word2[k]:
                key_word_vector2[i]+=1
                       
    x= key_word_vector1
    y= key_word_vector2
                       
                       
#     print(x)
#     print(y)

    nx= x / (np.sqrt(np.sum(x**2)) + 1e-8)
    ny =y / (np.sqrt(np.sum(y**2)) + 1e-8)
    cos_similarity_tfidf=float(np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y)))
#     cos_similarity= np.dot(nx, ny)
    return cos_similarity_tfidf

In [12]:
#以下为测试
df= pd.read_csv('ads_content_7224.csv')
df[df['ad_content'].str.contains('平安',na=False)]
test1 = df.iloc[731,0]
test2 = df.iloc[880,0]

print('计算两文本的余弦相似度为{}'.format(get_cos_similarity(test1,test2)))
print('基于TF-IDF计算为{}'.format(get_cos_similarity_tfidf(test1,test2)))
print('基于TextRankn计算为{}'.format(get_cos_similarity_textrank(test1,test2)))

计算两文本的余弦相似度为0.2897808849066627
基于TF-IDF计算为0.09999999999999998
基于TextRankn计算为0.13333333333333333
